In [1]:
from google.colab import drive
drive.mount("/content/drive")

%cd drive/MyDrive/tf-ssd-master/
!ls

Mounted at /content/drive
/content/drive/MyDrive/tf-ssd-master
augmentation.py  LICENSE  predictor.py	ssd_loss.py    trainer.py
environment.yml  logs	  __pycache__	trained        utils
__init__.py	 models   README.md	Trainer.ipynb


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
import augmentation
from ssd_loss import CustomLoss
from utils import bbox_utils, data_utils, io_utils, train_utils

In [4]:
from models import ssd_mobilenet_v2

In [5]:
batch_size = 32
epochs = 50 #CHANGED TO 50 BECAUSE IT RAN OUT OF RAM 
load_weights = False
with_voc_2012 = True
backbone = 'mobilenet_v2'
io_utils.is_valid_backbone(backbone)
#
if backbone == "mobilenet_v2":
    from models.ssd_mobilenet_v2 import get_model, init_model
else:
    from models.ssd_vgg16 import get_model, init_model
#
hyper_params = train_utils.get_hyper_params(backbone)

In [6]:
train_data, info = data_utils.get_dataset("voc/2007", "train+validation")
val_data, _ = data_utils.get_dataset("voc/2007", "test")
train_total_items = data_utils.get_total_item_size(info, "train+validation")
val_total_items = data_utils.get_total_item_size(info, "test")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/voc/2007/4.0.0.incomplete1HE162/voc-test.tfrecord


...

  0%|          | 0/2501 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/voc/2007/4.0.0.incomplete1HE162/voc-validation.tfrecord


  0%|          | 0/2510 [00:00<?, ? examples/s]

Dataset voc downloaded and prepared to /root/tensorflow_datasets/voc/2007/4.0.0. Subsequent calls will reuse this data.


In [7]:
if with_voc_2012:
    voc_2012_data, voc_2012_info = data_utils.get_dataset("voc/2012", "train+validation")
    voc_2012_total_items = data_utils.get_total_item_size(voc_2012_info, "train+validation")
    train_total_items += voc_2012_total_items
    train_data = train_data.concatenate(voc_2012_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/voc/2012/4.0.0.incomplete7179XV/voc-test.tfrecord


...

  0%|          | 0/5717 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/voc/2012/4.0.0.incomplete7179XV/voc-validation.tfrecord


  0%|          | 0/5823 [00:00<?, ? examples/s]

Dataset voc downloaded and prepared to /root/tensorflow_datasets/voc/2012/4.0.0. Subsequent calls will reuse this data.


In [8]:
labels = data_utils.get_labels(info)
labels = ["bg"] + labels
hyper_params["total_labels"] = len(labels)
img_size = hyper_params["img_size"]

In [9]:
train_data = train_data.map(lambda x : data_utils.preprocessing(x, img_size, img_size, augmentation.apply))
val_data = val_data.map(lambda x : data_utils.preprocessing(x, img_size, img_size))

In [10]:
data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()
train_data = train_data.shuffle(batch_size*4).padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)
val_data = val_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [11]:
ssd_model = get_model(hyper_params)
ssd_custom_losses = CustomLoss(hyper_params["neg_pos_ratio"], hyper_params["loc_loss_alpha"])
ssd_model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss=[ssd_custom_losses.loc_loss_fn, ssd_custom_losses.conf_loss_fn])
init_model(ssd_model)

9420800/9406464 [==============================] - 0s 0us/step


In [12]:
ssd_model_path = io_utils.get_model_path(backbone)
if load_weights:
    ssd_model.load_weights(ssd_model_path)
ssd_log_path = io_utils.get_log_path(backbone)

prior_boxes = bbox_utils.generate_prior_boxes(hyper_params["feature_map_shapes"], hyper_params["aspect_ratios"])
ssd_train_feed = train_utils.generator(train_data, prior_boxes, hyper_params)
ssd_val_feed = train_utils.generator(val_data, prior_boxes, hyper_params)

In [13]:
checkpoint_callback = ModelCheckpoint(ssd_model_path, monitor="val_loss", save_best_only=True, save_weights_only=True)
tensorboard_callback = TensorBoard(log_dir=ssd_log_path)
learning_rate_callback = LearningRateScheduler(train_utils.scheduler, verbose=0)

In [14]:
step_size_train = train_utils.get_step_size(train_total_items, batch_size)
step_size_val = train_utils.get_step_size(val_total_items, batch_size)
ssd_model.fit(ssd_train_feed,
              steps_per_epoch=step_size_train,
              validation_data=ssd_val_feed,
              validation_steps=step_size_val,
              epochs=epochs,
              callbacks=[checkpoint_callback, tensorboard_callback, learning_rate_callback])

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/50
518/518 [==============================] - 346s 647ms/step - loss: 6.0096 - loc_loss: 1.7673 - conf_loss: 4.2423 - val_loss: 37.1075 - val_loc_loss: 7.1265 - val_conf_loss: 29.9810 - lr: 0.0010
Epoch 2/50
518/518 [==============================] - 376s 727ms/step - loss: 4.7332 - loc_loss: 1.3947 - conf_loss: 3.3385 - val_loss: 16.2256 - val_loc_loss: 4.0828 - val_conf_loss: 12.1428 - lr: 0.0010
Epoch 3/50
518/518 [==============================] - 368s 710ms/step - loss: 4.3835 - loc_loss: 1.2890 - conf_loss: 3.0945 - val_loss: 11.9503 - val_loc_loss: 2.9160 - val_conf_loss: 9.0343 - lr: 0.0010
Epoch 4/50
518/518 [==============================] - 321s 620ms/step - loss: 4.2288 - loc_loss: 1.2467 - conf_loss: 2.9821 - val_loss: 12.4689 - val_loc_loss: 3.0527 - val_conf_loss: 9.4162 - lr: 0.0010
Epoch 5/50
518/518 [==============================] - 316s 610ms/step - loss: 4.0939 - loc_loss: 1.2026 - conf_loss: 2.8913 - val_loss: 10.1011 - val_loc_loss: 2.6416 - val_conf_loss